In [ ]:
!pip install --upgrade tensorflow==1.15

     |████████████████████████████████| 412.3MB 28kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.8MB 29.2MB/s 
     |████████████████████████████████| 512kB 31.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=d85ee670b5b4d7f808386e0c17ec41b388d3387a3fc2ddcc345b804743b02b28
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensor

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

#Load Dataset

In [ ]:
df = pd.read_csv('pulsar_stars.csv')
df

FileNotFoundError: ignored

In [ ]:
x_data = df[['Mean of the integrated profile','Standard deviation of the integrated profile','Excess kurtosis of the integrated profile','Skewness of the integrated profile','Mean of the DM-SNR curve','Standard deviation of the DM-SNR curve','Excess kurtosis of the DM-SNR curve','Skewness of the DM-SNR curve']]
y_data = df[['target_class']]

#Preprocessing Dataset

In [ ]:
scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

encoder = OneHotEncoder()
y_data = encoder.fit_transform(y_data).toarray()

#Declare Variable

In [ ]:
layer = {
    'input' : 8,
    'hidden' : 3,
    'output' : 2
}

weight = {
    'th' : tf.Variable(tf.random.normal([layer['input'],layer['hidden']])),
    'to' : tf.Variable(tf.random.normal([layer['hidden'],layer['output']]))
}

bias = {
    'th' : tf.Variable(tf.random.normal([layer['hidden']])),
    'to' : tf.Variable(tf.random.normal([layer['output']]))
}


#Split Dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
x = tf.placeholder(tf.float32,[None, layer['input']])
target = tf.placeholder(tf.float32,[None, layer['output']])

#Function Forward Pass

In [ ]:
def forward_pass():
    wx_b1 = tf.matmul(x,weight['th']) + bias['th']
    y1 = tf.nn.softmax(wx_b1)

    wx_b2 = tf.matmul(y1,weight['to']) + bias['to']
    y2 = tf.nn.sigmoid(wx_b2)

    return y2

#isi value

In [ ]:
y = forward_pass()
epoch = 10
alpha = 0.6
#Mean Squared Error Function
error = tf.reduce_mean(0.5 * (target-y)**2)
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(error)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoch):
        sess.run(
            train,
            feed_dict={
                x: x_train,
                target: y_train
            })
        if i % 1 == 0:
            current_error = sess.run(
                error,
                feed_dict={
                    x: x_train,
                    target: y_train
                })
            true_prediction = tf.equal(tf.argmax(y,axis=1),tf.argmax(target,axis=1))
            accuracy = tf.reduce_mean(tf.cast(true_prediction,tf.float32))
            accuracy = sess.run(
                accuracy,
                feed_dict={
                    x: x_test,
                    target: y_test
                })
            print(f'EPOCH:{i+1} | ACCURACY: {accuracy:.4f} | ERRROR: {d}')

